## Fetch Option chain data from NSE web site. Part-1


### Import necessry library

In [ ]:
import requests
import json
import pandas as pd

: 

#### Set url tp scrap option chain data 
As of now here symbol is set as 'NIFTY' , this can be change for other symbpls as well e.g FINNIFTY or BANKNIFTY etc.
header setting also require else response will hang

In [ ]:
url = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
headers={'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

: 

#### Using requests library, get the data. 
Parameters are url and header. timeout is set to 10 seconds in case due to network issue there is delay in response 

In [ ]:
response = requests.get(url, headers=headers, timeout=10)

response_text=response.text
json_object = json.loads(response_text)

: 

In [ ]:
json_object

: 

## Fetch Option chain data from NSE web site. Part-2

#### Set the expiry date

In [ ]:
expiry_date = '31-Dec-2026'

: 

In [ ]:
def set_decimal(x):
    return ('%.2f' % x).rstrip('0').rstrip('.')

: 

In [ ]:
set_decimal(10.234567)

: 

In [ ]:
data = json_object['records']['data']

: 

In [ ]:
len(data)

: 

In [ ]:
e_date=json_object['records']['expiryDates']

: 

In [ ]:
e_date

: 

In [ ]:
oc_data = {}
for ed in e_date:
    oc_data[ed]={"CE":[], "PE":[]}
    for di in range(len(data)):
        
        if data[di]['expiryDate'] == ed:
            if 'CE' in data[di].keys() and data[di]['CE']['expiryDate'] == ed:                
                oc_data[ed]["CE"].append(data[di]['CE'])
            else:
                oc_data[ed]["CE"].append('-')

            if 'PE' in data[di].keys() and data[di]['PE']['expiryDate'] == ed:
                oc_data[ed]["PE"].append(data[di]['PE'])
                
            else:
                oc_data[ed]["PE"].append('-')

: 

In [ ]:
oc_data

: 

In [ ]:
keys_to_exclude = [
                    'pchangeinOpenInterest', 
                    'totalBuyQuantity', 
                    'totalSellQuantity', 
                    'underlyingValue',
                    'expiryDate',
                    'underlying',
                    'identifier',
                    'pChange'
]

: 

In [ ]:
oc_data_dt=oc_data[expiry_date]

CE=list(oc_data_dt['CE'])
PE=list(oc_data_dt['PE'])


: 

In [ ]:
for i in range(len(CE)):
    if CE[i] != '-':
        for key in keys_to_exclude:
            del CE[i][key]

    if PE[i] != '-':
        for key in keys_to_exclude:
            del PE[i][key]


: 

In [ ]:
CE[0]

: 

In [ ]:
PE[0]

: 

In [ ]:
l_OC = []

for i in range(len(CE)):
    l_CE=[]
    l_PE=[]

    if CE[i] != '-':
        sp = CE[i]['strikePrice']
        l_CE =  [  
                    CE[i]['openInterest'], 
                    CE[i]['changeinOpenInterest'] , 
                    CE[i]['totalTradedVolume'] , 
                    CE[i]['impliedVolatility'] , 
                    CE[i]['lastPrice'] , 
                    set_decimal(CE[i]['change']), 
                    CE[i]['bidQty']   , 
                    CE[i]['bidprice'] , 
                    CE[i]['askPrice'] ,
                    CE[i]['askQty'] , 
                    CE[i]['strikePrice']
                ]
    else:
        sp = PE[i]['strikePrice']
        l_CE= list(['-','-','-','-','-','-','-','-','-','-',sp])

    if PE[i] != '-':
        l_PE =  [
                    PE[i]['bidQty']   ,
                    PE[i]['bidprice'] ,
                    PE[i]['askPrice'] ,
                    PE[i]['askQty'] , 
                    set_decimal(PE[i]['change']),
                    PE[i]['lastPrice'] ,
                    PE[i]['impliedVolatility'] ,
                    PE[i]['totalTradedVolume'] ,
                    PE[i]['changeinOpenInterest'] ,
                    PE[i]['openInterest']
        ] 

    else:
        l_PE = list(['-','-','-','-','-','-','-','-','-','-'])
    
    
    l_OC_t = l_CE + l_PE
    l_OC_t[:] = [x if x != 0 else '-' for x in l_OC_t]

    l_OC.append(l_OC_t)


: 

In [ ]:
l_OC

: 

In [ ]:
OC_col = ['c_OI', 'c_CHNG_IN_OI', 'c_VOLUME', 'c_IV', 'c_LTP', 'c_CHNG', 'c_BID_QTY', 'c_BID', 'c_ASK', 'c_ASK_QTY',  'STRIKE', 'p_BID_QTY', 'p_BID', 'p_ASK', 'p_ASK_QTY',  'p_CHNG', 'p_LTP', 'p_IV', 'p_VOLUME', 'p_CHNG_IN_OI', 'p_OI'] 

: 

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df = pd.DataFrame(l_OC)
df.columns = OC_col

: 

In [ ]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

: 

In [ ]:
df

: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 
# Read the CSV file into a DataFrame
df = pd.read_csv('option_chain_data.csv')
 
# Drop rows with missing values
df = df.dropna()
 
# Convert columns to numeric
numeric_cols = ['c_OI', 'c_CHNG_IN_OI', 'c_VOLUME', 'c_IV', 'c_LTP', 'c_CHNG', 'c_BID_QTY', 'c_BID', 'c_ASK', 'c_ASK_QTY', 'p_BID_QTY', 'p_BID', 'p_ASK', 'p_ASK_QTY', 'p_CHNG', 'p_LTP', 'p_IV', 'p_VOLUME', 'p_CHNG_IN_OI', 'p_OI']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
 
# Plot the graph
plt.figure(figsize=(14, 7))
plt.plot(df['STRIKE'], df['c_OI'], label='Call Open Interest', marker='o')
plt.plot(df['STRIKE'], df['p_OI'], label='Put Open Interest', marker='o')
plt.xlabel('Strike Price')
plt.ylabel('Open Interest')
plt.title('Open Interest Analysis')
plt.legend()
plt.grid(True)
plt.show()

: 

In [ ]:
import plotly.express as px
 
# Create a long format DataFrame for Plotly
df_long = df.melt(id_vars=['STRIKE'], var_name='Metric', value_name='Value')
 
# Plot the graph
fig = px.line(df_long, x='STRIKE', y='Value', color='Metric', title='Open Interest Analysis')
fig.show()

: 